In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'VersiColor', 'Virginica']

In [ ]:
train_path = tf.keras.utils.get_file('iris_training.csv',"https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file('iris_test.csv',"https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header=0)

573/573 [==============================] - 0s 0us/step


In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train.shape

(120, 5)

In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [ ]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [ ]:
train.keys()

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')

In [ ]:
def input_fn(features, labels, training=True, batch_size = 256):
  #Convert the inputs to da Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #Shuffle and repeat for training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [ ]:
#Feature columns
feature_columns = []
for key in train.keys():
  feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_columns,
    hidden_units = [30,10],
    n_classes =3
)

In [ ]:
classifier.train(
    input_fn = lambda: input_fn(train, train_y, training = True),
    steps =5000
)

Instructions for updating:
Use standard file utilities to get mtimes.


Evaluation

In [ ]:
eval_result = classifier.evaluate(
    input_fn = lambda: input_fn(test, test_y, training = False)
)

print('\n Test  set Accuracy: {accuracy: 0.3f}\n'.format(**eval_result))


 Test  set Accuracy:  0.933



Predition

In [ ]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [ ]:
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ":")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(
      SPECIES[class_id], 100 * probability
  ))

Please type numeric values as prompted.
SepalLength:6.9
SepalWidth:3.1
PetalLength:5.4
PetalWidth:2.1
Prediction is "Virginica" (65.4%)
